In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator
import tensorflow_addons as tfa
from functools import partial
from numpy.random import default_rng
import matplotlib.pyplot as plt
rng = default_rng()


In [ ]:
SEED = 42
PROJECT_FOLDER = "../hotel_recog_splitted/bed"
TRAIN_DATA_FOLDER = PROJECT_FOLDER

IMAGE_SIZE = (224, 224)
input_size = 224
VAL_SPLIT = 0.1

In [ ]:
train_ds = image_dataset_from_directory(
    directory=TRAIN_DATA_FOLDER,
    labels='inferred',
    label_mode='categorical',
    batch_size=10,
    seed=1337,
    image_size=IMAGE_SIZE,
    crop_to_aspect_ratio=True,
    subset='training',
    validation_split=VAL_SPLIT
)

val_ds = image_dataset_from_directory(
    directory=TRAIN_DATA_FOLDER,
    labels='inferred',
    label_mode='categorical',
    batch_size=10,
    seed=1337,
    image_size=IMAGE_SIZE,
    crop_to_aspect_ratio=True,
    subset='validation',
    validation_split=VAL_SPLIT
)

# test_ds = image_dataset_from_directory(
#     directory=TEST_DATA_FOLDER,
#     labels=None,
#     batch_size=1,
#     seed=1337,
#     image_size=IMAGE_SIZE,
#     crop_to_aspect_ratio=True,
# )


In [ ]:
n_classes = len(train_ds.class_names)
n_classes

In [ ]:
def augment_using_ops(images, labels):
    images = tf.image.random_flip_left_right(images)
    images = tf.image.random_brightness(images, 0.2)
    images = tf.image.random_contrast(images, 0.1, 0.6)
    return (images, labels)

train_ds_aug = train_ds.map(augment_using_ops)
val_ds_aug = train_ds.map(augment_using_ops)

In [ ]:
def random_cutout(images_tensor, labels):
    ratios_x = rng.uniform(0.1, 0.4)*images_tensor.shape[1]
    ratios_y = rng.uniform(0.1, 0.4)*images_tensor.shape[2]

    xs_mask = tf.cast(ratios_x, tf.int32)
    xs_mask = xs_mask if xs_mask % 2 == 0 else xs_mask+1 # force number even

    ys_mask = tf.cast(ratios_y, tf.int32)
    ys_mask = ys_mask if ys_mask % 2 == 0 else ys_mask+1 # force number even

    return tfa.image.random_cutout(images_tensor, (xs_mask, ys_mask), constant_values = (255.0, 0.0, 0.0)), labels

train_ds_aug = train_ds_aug.map(random_cutout)
val_ds_aug = val_ds_aug.map(random_cutout)

In [ ]:
a = next(iter(train_ds_aug))

In [ ]:
plt.imshow(a[0][0]/255.0)

# Models

In [ ]:
def create_finetuned_efficientnet_model(n_classes):
    efficientnet_layer = tf.keras.applications.EfficientNetB0(
            include_top= False,
            weights="imagenet")
    efficientnet_layer.trainable = False
    
    model = keras.Sequential([
        efficientnet_layer,
        keras.layers.GlobalAveragePooling2D(name = "avg_pool"),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(n_classes, activation = 'softmax')
    ])
    
    model.compile(
        optimizer= tf.keras.optimizers.Adam(learning_rate=1e-2),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [ ]:
def create_unfreezed_efficientnet_model(model):
    for layer in model.layers[-35:]:
        if not isinstance(layer, keras.layers.BatchNormalization):
            layer.trainable = True
    
    model.compile(
        optimizer= tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [ ]:
model = create_finetuned_efficientnet_model(n_classes)
model.summary()

In [ ]:
callbacks = [
    tf.keras.callbacks.TensorBoard(
        log_dir = 'logs'
    ), 
    tf.keras.callbacks.ModelCheckpoint(
        filepath='ckp',
        save_weights_only=True,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
]

In [ ]:
history = model.fit(train_ds_aug, epochs=10, callbacks=callbacks, validation_data=val_ds_aug)
model.save('../models/bedroom_finetune_part1.h5')

In [ ]:
model = tf.keras.models.load_model('../models/bedroom_finetune_part1.h5')
model.load_weights('ckp')

model = create_unfreezed_efficientnet_model(model)
history2 = model.fit(train_ds_aug, epochs=20, callbacks=callbacks, validation_data=val_ds_aug)
model.save('../models/bedroom_finetune_part2.h5')

In [ ]:
metrics = history.history
plt.plot(history.epoch, metrics['loss'], metrics['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.show()

In [ ]:
metrics = history2.history
plt.plot(history2.epoch, metrics['loss'], metrics['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.show()